# Pinocchio Geometry Examples 

## 1. setup

In [1]:
import pinocchio as pin
from utils import MeshcatVisualizer

load the robot and visualize it:

In [2]:
# This is distributed with the robotpkg-pinocchio package
robot_path = "/opt/openrobots/share/example-robot-data"
urdf_filename = robot_path + "/robots/panda_description/urdf/panda.urdf"
stl_filename = robot_path + "/robots/panda_description/meshes/"

# Load the urdf model
model, collision_model, visual_model = pin.buildModelsFromUrdf(urdf_filename, "/opt/openrobots/share")

# visualize the robot
viz = MeshcatVisualizer(model=model, collision_model=collision_model, visual_model=visual_model)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


## Some debugging information

In [3]:
q = pin.neutral(model)
viz.display(q)
viz.displayCollisions(True)

We can see some figures here:

<img src="image/urdf_meshcat.jpg" style="width:600px;">

Collisions and visuals are independent, we can notice collision is more than visuals. This exactly matches to the URDF of fingers one visual have one visual but multiple collisions:

```XML
<link name="panda_rightfinger">
    <visual>
        <origin rpy="0 0 3.141592653589793" xyz="0 0 0"/>
        <geometry>
            <mesh filename="package://example-robot-data/robots/panda_description/meshes/visual/finger.dae" />
        </geometry>
    </visual>
    <!-- screw mount -->
    <collision>
        <origin rpy="0 0 0" xyz="0 -18.5e-3 11e-3"/>
        <geometry>
            <box size="22e-3 15e-3 20e-3"/>
        </geometry>
    </collision>
    <!-- cartriage sledge -->
    <collision>
        <origin rpy="0 0 0" xyz="0 -6.8e-3 2.2e-3"/>
        <geometry>
            <box size="22e-3 8.8e-3 3.8e-3"/>
        </geometry>
    </collision>
    <!-- And many ... -->
</link>
```

In [4]:
C = collision_model.geometryObjects.tolist()
F = model.frames.tolist()
V = visual_model.geometryObjects.tolist()
J = model.joints.tolist()

Lets see some of the important data, the geometry objects and frames that matches to visualization.
![C_F_V_objects](image/C_F_V_objects.jpg)

They are lists of objects containing the collision, frame and visual information. 

For [GeometryObject](https://gepettoweb.laas.fr/doc/stack-of-tasks/pinocchio/master/doxygen-html/structpinocchio_1_1GeometryObject.html) object, they are either mesh or primitives like this:

![C_mesh_primitives](image/C_mesh_primitives.jpg)

## 2 Distance Calculation

In [5]:
collision_model.addAllCollisionPairs()
print("num collision pairs - initial:", len(collision_model.collisionPairs))
for i in range(len(collision_model.collisionPairs)):
    print("id = ", i, " : ", collision_model.collisionPairs[i].first, " - ", collision_model.collisionPairs[i].second)

num collision pairs - initial: 123
id =  0  :  0  -  1
id =  1  :  0  -  2
id =  2  :  0  -  3
id =  3  :  0  -  4
id =  4  :  0  -  5
id =  5  :  0  -  6
id =  6  :  0  -  7
id =  7  :  0  -  8
id =  8  :  0  -  9
id =  9  :  0  -  10
id =  10  :  0  -  11
id =  11  :  0  -  12
id =  12  :  0  -  13
id =  13  :  0  -  14
id =  14  :  0  -  15
id =  15  :  0  -  16
id =  16  :  1  -  2
id =  17  :  1  -  3
id =  18  :  1  -  4
id =  19  :  1  -  5
id =  20  :  1  -  6
id =  21  :  1  -  7
id =  22  :  1  -  8
id =  23  :  1  -  9
id =  24  :  1  -  10
id =  25  :  1  -  11
id =  26  :  1  -  12
id =  27  :  1  -  13
id =  28  :  1  -  14
id =  29  :  1  -  15
id =  30  :  1  -  16
id =  31  :  2  -  3
id =  32  :  2  -  4
id =  33  :  2  -  5
id =  34  :  2  -  6
id =  35  :  2  -  7
id =  36  :  2  -  8
id =  37  :  2  -  9
id =  38  :  2  -  10
id =  39  :  2  -  11
id =  40  :  2  -  12
id =  41  :  2  -  13
id =  42  :  2  -  14
id =  43  :  2  -  15
id =  44  :  2  -  16
id =  45 

In [6]:
srdf_filename = robot_path + "/robots/panda_description/srdf/panda.srdf"
import os
srdf_filename = os.getcwd() + "/panda-alternative.srdf"

pin.removeCollisionPairs(model, collision_model, srdf_filename)
print("num collision pairs - after srdf removal:", len(collision_model.collisionPairs))
for i in range(len(collision_model.collisionPairs)):
    print("id = ", i, " : ", collision_model.collisionPairs[i].first, " - ", collision_model.collisionPairs[i].second)

num collision pairs - after srdf removal: 12
id =  0  :  0  -  5
id =  1  :  0  -  6
id =  2  :  0  -  7
id =  3  :  0  -  8
id =  4  :  1  -  5
id =  5  :  1  -  6
id =  6  :  1  -  7
id =  7  :  1  -  8
id =  8  :  2  -  5
id =  9  :  2  -  6
id =  10  :  2  -  7
id =  11  :  2  -  8


In [7]:
data = model.createData()
collision_data = pin.GeometryData(collision_model)

In [8]:
#pin.computeCollisions(model, data, collision_model, collision_data, q)

In [9]:
#pin.updateGeometryPlacements(model, data, collision_model, collision_data, q)
#pin.computeDistances(collision_model, collision_data)

collision_model (GeometryModel): 
- geometryObjects: an vector containing information about each vector
  - See above ...
  - parent joint: int, the parent joint index
  - parent frame: int, 
  - placement: placement between the parent joint frame and the current frame
- collisionPairs: an vector indicating which pairs are in collision
  - first: 0
  - second: 5
- collisionPairMapping: an Matrix indicating the contact information, similar as collisionPairs
  - (row 0) [-1, -1, -1, -1, -1, 0, ...] 
  - (row 5) [0, ...] 
  - (Object 0 and 5 has CollisionPair 0)

collision_distance (GeometryData, getting from ): 
- activeCollsionPairs: a bool vector indicating if each collision pair is activated
- collisionRequests: ... irrelevant
- collisionResults: ... irrelevant
- collision_functors: ... irrelevant
- distanceRequests: something for coal
- distanceResults: The data structure that saves the distance Calculation Result
  - min_dustance: minimum distance
  - nearest_points: an array that has 2 points indicating the contacts
- distance_functors: something for coal
- oMg: The frame position of each geometryObject
- radius: ... empty?

updateGeometryPlacements: GeometryData.oMg[i] is updated from Data.oMi of parent joint (if avaliable) and GeometryObject[i].placement

<img src="image/updateGeometry.jpg" style="width:600px;">

computeDistances: use for each [i] GeometryData.oMg[first/second], Geometry.distance_functors to update GeometryData.distance_request and GeometryData.distance_result

<img src="image/computeDistance.jpg" style="width:600px;">

# 3 Make up our obstacles as new collision consideration

In [ ]:
# remove all collision pairs to ignore the collisions
geom_model = pin.GeometryModel(collision_model)
collision_model.removeAllCollisionPairs()


In [12]:
# debug this line to view variables
a = 0